#### These techs are based on the book *Deep Learning with Python*.

In [1]:
import numpy as np
import pandas as pd
import keras 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# load data, and simply norm
raw_data = pd.read_csv("../dataset/MNIST/train.csv")
data = raw_data.iloc[:,1:].values
labels = keras.utils.to_categorical(raw_data["label"])
train_images, test_images, train_labels, test_labels = train_test_split(data, labels)
train_images = train_images / 255
test_images = test_images / 255

### Naive Conv Layer

In [3]:
# first reshape images to 2D image
train_images = train_images.reshape(train_images.shape[0], 28, 28, -1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, -1)
print(train_images.shape, test_images.shape)

(31500, 28, 28, 1) (10500, 28, 28, 1)


In [18]:
def naive_conv(input_shape=(28, 28, 1)):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [19]:
# hyper params
epochs = 20
batch_size = 128

In [20]:
naive_conv = naive_conv()
naive_hist = naive_conv.fit(train_images, train_labels, epochs=epochs,
                           batch_size=batch_size)

Epoch 1/20
31500/31500 [==============================] - 2s 77us/step - loss: 0.3377 - acc: 0.8966
Epoch 2/20
31500/31500 [==============================] - 2s 51us/step - loss: 0.1116 - acc: 0.9656
Epoch 3/20
31500/31500 [==============================] - 2s 51us/step - loss: 0.0760 - acc: 0.9767
Epoch 4/20
31500/31500 [==============================] - 2s 51us/step - loss: 0.0625 - acc: 0.9803
Epoch 5/20
31500/31500 [==============================] - 2s 51us/step - loss: 0.0555 - acc: 0.9833
Epoch 6/20
31500/31500 [==============================] - 2s 51us/step - loss: 0.0466 - acc: 0.9849
Epoch 7/20
31500/31500 [==============================] - 2s 50us/step - loss: 0.0413 - acc: 0.9873
Epoch 8/20
31500/31500 [==============================] - 2s 51us/step - loss: 0.0376 - acc: 0.9884
Epoch 9/20
31500/31500 [==============================] - 2s 51us/step - loss: 0.0343 - acc: 0.9892
Epoch 10/20
31500/31500 [==============================] - 2s 51us/step - loss: 0.0297 - acc: 0.9909

In [21]:
_, acc = naive_conv.evaluate(test_images, test_labels)
print(acc)

10500/10500 [==============================] - 1s 109us/step
0.9911428571428571
